# Kaggle progect
https://www.kaggle.com/c/intel-mobileodt-cervical-cancer-screening/data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

C:\Users\Hradi\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import cv2
import math
from sklearn.utils import shuffle
from glob import glob
import os
from functools import partial
import tensorflow as tf

In [3]:
TRAIN_DATA = r"D:\Nastya\SelfProgects\My-projects\DataScience"

types = ['Type_1','Type_2','Type_3']
type_ids = []

for type in enumerate(types):
    type_i_files = glob(os.path.join(TRAIN_DATA, type[1], "*.jpg"))
    type_i_ids = np.array([s[len(TRAIN_DATA)+8:-4] for s in type_i_files])
    type_ids.append(type_i_ids)

In [ ]:
def get_filename(image_id, image_type):
    """
    Method to get image file path from its id and type   
    """
    if image_type == "Type_1" or \
        image_type == "Type_2" or \
        image_type == "Type_3":
        data_path = os.path.join(TRAIN_DATA, image_type)
    else:
        raise Exception("Image type '%s' is not recognized" % image_type)

    ext = 'jpg'
    return os.path.join(data_path, "{}.{}".format(image_id, ext))

def get_image_data(image_id, image_type):
    """
    Method to get image data as np.array specifying image id and type
    """
    fname = get_filename(image_id, image_type)
    img = cv2.imread(fname)
    assert img is not None, "Failed to read image : %s, %s" % (image_id, image_type)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def get_image(image_id, image_type, bw=1):
    """
    Method to get image data as np.array specifying image id and type
    """
    fname = get_filename(image_id, image_type)
    img = cv2.imread(fname, bw)
    assert img is not None, "Failed to read image : %s, %s" % (image_id, image_type)
    if bw:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [ ]:
# Without photos example because medical data is privacy
data = []
lables = []
for i in range(len(type_ids)):
    for im_id in range(len(type_ids[i])):
        im = get_image(type_ids[i][im_id], 'Type_'+str(i+1))
        im_bw =  get_image(type_ids[i][im_id], 'Type_'+str(i+1), 0)
        vfunc = np.vectorize(lambda x, y: np.array([x-y]) if x-y>=0 else np.array([0]))
        dim = (64, 64)
        im = cv2.resize(im, dim)
        red = im[:, :, 2]
        bw = cv2.resize(im_bw, dim)
        res = vfunc(red, bw)
        data.append(res)
        lables.append(i+1)
data = np.array(data, dtype=float)/255
data = data.reshape((len(data), 64, 64, 1))
lables = np.array(lables)-1

In [6]:
def deformation(X_input):
    X_output = np.zeros_like(X_input)
    for i in range(len(X_input)):
        M = cv2.getRotationMatrix2D((32,32), (30 *(np.random.rand()-.5)), 1)
        M[0,2] = 4 *(np.random.rand()-.5)
        M[1,2] = 4 *(np.random.rand()-.5)
        imgswap = cv2.warpAffine(X_input[i], M, (64,64))
        trans = 2*(np.random.rand()-.5)
        pts1 = np.float32([[2,2],[30,2],[0,30],[30,30]])
        pts2 = np.float32([[trans,trans],[30+trans,trans],[trans,30+trans],[30+trans,30+trans]])
        M = cv2.getPerspectiveTransform(pts1,pts2)
        X_output[i] = cv2.warpPerspective(imgswap,M,(64,64)).reshape((64, 64, 1))
    return X_output

In [8]:
type_1 = data[np.where(lables == 1)]
type_2 = data[np.where(lables == 2)]
type_3 = data[np.where(lables == 3)]
typs = [type_1, type_2, type_3]
dat = data
lable = lables
p = [17, 5, 9]
for i in range(3):
    for j in range(p[i]):
        dat = np.concatenate([dat, deformation(typs[i])])
        lable = np.concatenate([lable, lables[np.where(lables == i+1)]])
print(len(lable))

17008


In [9]:
x_train, x_test, y_train, y_test = train_test_split(dat, lable, test_size=0.33)
x_test, x_valid, y_test, y_valid = train_test_split(x_test, y_test, test_size=0.33)

In [10]:
def Model(x):    

    mu = 0
    sigma = 0.1
    keep_prob = 0.9
    strides_conv = [1, 1, 1, 1]
    strides_pool = [1, 2, 2, 1]
    
    #________________________________Layer 1__________________________________________________

    # Convolutional. Input = 64x50x1. Filter = 5x5x1. Output = 60x46x6.
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    conv1_W = tf.Variable(tf.random_normal(shape=(5, 5, 1, 6), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(6))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=strides_conv, padding='VALID') + conv1_b

    # Apply activation function
    conv1 = tf.nn.relu(conv1)
    #conv1 = tf.nn.dropout(conv1, keep_prob)

    #________________________________Layer 2__________________________________________________
    
     # Convolutional. Input = 60x28x6. Filter = 3x3x6. Output = 14x14x6.
    conv2_W = tf.Variable(tf.random_normal(shape=(3, 3, 6, 12), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(12))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=strides_conv, padding='SAME') + conv2_b

    # Pooling. Input = 60x28x6. Output = 30x14x6.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    #________________________________Layer 3__________________________________________________

    # Convolutional. Input = 30x14x6. Filter = 5x5x12. Output = 26x10x16.
    conv3_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 12, 16), mean = mu, stddev = sigma))
    conv3_b = tf.Variable(tf.zeros(16))
    conv3   = tf.nn.conv2d(conv2, conv3_W, strides=strides_conv, padding='VALID') + conv3_b
    
    # Apply activation function
    conv3 = tf.nn.relu(conv3)

    # Pooling. Input = 26x10x16. Output = 13x5x16.
    conv3 = tf.nn.max_pool(conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    #________________________________Layer 4__________________________________________________
    
    # Flatten. Input = 13x5x16. Output = 400.
    fc0   = tf.reshape(conv3, [-1, int(13*13*16)])
    fc0 = tf.nn.dropout(fc0, keep_prob)
    
    # Fully Connected. Input = 400. Output = 120.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(13*13*16, 120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    
    # Apply activation function
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, keep_prob)

    #________________________________Layer 5__________________________________________________
        
    # Fully Connected. Input = 120. Output = 84.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(84))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
    # Apply activation function
    fc2  = tf.nn.relu(fc2)
    fc2  = tf.nn.dropout(fc2, keep_prob)

    #________________________________Layer 6__________________________________________________
    
    # Fully Connected. Input = 84. Output = 43.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, 3), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(3))
    logits = tf.matmul(fc2, fc3_W) + fc3_b
    
    return logits

In [11]:
x = tf.placeholder(tf.float32, (None, 64, 64, 1))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 3)

In [12]:
rate = 0.001

logits = Model(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [13]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

BATCH_SIZE = 100

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [15]:
from time import time
start_time = time()

EPOCHS = 45

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(x_train)

    print("Training...")
    print()
    for i in range(EPOCHS):
        X_shuff, y_shuff = shuffle(x_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_shuff[offset:end], y_shuff[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})

        validation_accuracy = evaluate(x_valid, y_valid)
        
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()

    saver.save(sess, "./trained_variables.ckpt")
    print("Model saved")    
    print("Final Validation Accuracy = {:.3f}".format(validation_accuracy))

total_time = time() - start_time
minutes, seconds = divmod(total_time, 60)
print ("Total time for model training: ", minutes, "min, {:.0f}".format(seconds),  "s ")

Training...

EPOCH 1 ...
Validation Accuracy = 0.815

EPOCH 2 ...
Validation Accuracy = 0.815

EPOCH 3 ...
Validation Accuracy = 0.815

EPOCH 4 ...
Validation Accuracy = 0.815

EPOCH 5 ...
Validation Accuracy = 0.815

EPOCH 6 ...
Validation Accuracy = 0.817

EPOCH 7 ...
Validation Accuracy = 0.818

EPOCH 8 ...
Validation Accuracy = 0.821

EPOCH 9 ...
Validation Accuracy = 0.823

EPOCH 10 ...
Validation Accuracy = 0.832

EPOCH 11 ...
Validation Accuracy = 0.832

EPOCH 12 ...
Validation Accuracy = 0.838

EPOCH 13 ...
Validation Accuracy = 0.851

EPOCH 14 ...
Validation Accuracy = 0.865

EPOCH 15 ...
Validation Accuracy = 0.859

EPOCH 16 ...
Validation Accuracy = 0.863

EPOCH 17 ...
Validation Accuracy = 0.866

EPOCH 18 ...
Validation Accuracy = 0.869

EPOCH 19 ...
Validation Accuracy = 0.858

EPOCH 20 ...
Validation Accuracy = 0.887

EPOCH 21 ...
Validation Accuracy = 0.880

EPOCH 22 ...
Validation Accuracy = 0.881

EPOCH 23 ...
Validation Accuracy = 0.887

EPOCH 24 ...
Validation Accura